In [1]:
%%capture
! pip install -q --upgrade llama-index pypdf llama-index llama-index-embeddings-huggingface sentence-transformers llama-index-llms-llama-cpp llama-index-vector-stores-chroma

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,StorageContext
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from IPython.display import Markdown, display

In [4]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-21 09:18:19--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  42.5MB/s    in 0.4s    

2024-06-21 09:18:19 (42.5 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [5]:
llm  = LlamaCPP(
   model_url='https://huggingface.co/bartowski/Llama-3-8B-Instruct-Gradient-1048k-GGUF/resolve/main/Llama-3-8B-Instruct-Gradient-1048k-Q5_K_S.gguf',
   model_path=None,
   temperature=0.1,
   max_new_tokens=256,
   context_window=3900,
   generate_kwargs={},
   model_kwargs={"n_gpu_layers":-1},
   verbose=True
)

total size (MB): 5599.29


5340it [01:01, 86.18it/s]                           
llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from /tmp/llama_index/models/Llama-3-8B-Instruct-Gradient-1048k-Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Llama-3-8B-Instruct-Gradient-1048k
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 1048576
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.at

In [6]:
documents = SimpleDirectoryReader(input_files=['metagpt.pdf']).load_data()
print((documents[0].text))

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, are complicated through logic inconsistencies due to
cascading hallucinations caused by naively chaining LLMs. Here we introduce
MetaGPT, an innovative meta-programm

In [7]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

# Construct vector store
# set up ChromaVectorStore and load in data
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Naive RAG

In [8]:
query_str = "How do agents share information with other agents?"

In [10]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))


llama_print_timings:        load time =  334620.25 ms
llama_print_timings:      sample time =     171.37 ms /    60 runs   (    2.86 ms per token,   350.11 tokens per second)
llama_print_timings: prompt eval time =  978194.86 ms /  1488 tokens (  657.39 ms per token,     1.52 tokens per second)
llama_print_timings:        eval time =   65953.22 ms /    59 runs   ( 1117.85 ms per token,     0.89 tokens per second)
llama_print_timings:       total time = 1044477.67 ms /  1547 tokens


<b> Agents use a shared message pool that allows them to exchange messages directly. They can publish their structured messages in the pool but also access messages from other entities transparently. Any agent can retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses.</s></b>

## Using HyDE Transform

In [13]:
hyde = HyDEQueryTransform(llm=llm,include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit

llama_print_timings:        load time =  334620.25 ms
llama_print_timings:      sample time =     499.98 ms /   186 runs   (    2.69 ms per token,   372.01 tokens per second)
llama_print_timings: prompt eval time =   20695.10 ms /    32 tokens (  646.72 ms per token,     1.55 tokens per second)
llama_print_timings:        eval time =  187267.83 ms /   185 runs   ( 1012.26 ms per token,     0.99 tokens per second)
llama_print_timings:       total time =  208913.76 ms /   217 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =  334620.25 ms
llama_print_timings:      sample time =     153.95 ms /    60 runs   (    2.57 ms per token,   389.74 tokens per second)
llama_print_timings: prompt eval time =  972534.49 ms /  1487 tokens (  654.02 ms per token,     1.53 tokens per second)
llama_print_timings:        eval time =   59543.13 ms /    59 runs   ( 1009.21 ms per token,     0.99 tokens per second)
llama_print_timings:       to

<b> Agents use a shared message pool that allows them to exchange messages directly. They can publish their structured messages in the pool but also access messages from other entities transparently. Any agent can retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses.</s></b>

In [14]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

Llama.generate: prefix-match hit

llama_print_timings:        load time =  334620.25 ms
llama_print_timings:      sample time =     375.23 ms /   146 runs   (    2.57 ms per token,   389.10 tokens per second)
llama_print_timings: prompt eval time =   19064.38 ms /    32 tokens (  595.76 ms per token,     1.68 tokens per second)
llama_print_timings:        eval time =  148115.00 ms /   145 runs   ( 1021.48 ms per token,     0.98 tokens per second)
llama_print_timings:       total time =  167799.87 ms /   177 tokens


'The agents are able to communicate with each other through a shared database. This allows them to quickly and easily access information about their fellow agents, including their location, skills, and availability. The database also includes information about potential clients, such as their needs and preferences.\n\nIn addition to the database, agents can communicate directly with each other through various channels. For example, they may use email or instant messaging to discuss a client\'s needs and coordinate efforts. They may also attend meetings or conferences where they can network and share ideas.\n\nOverall, communication is key to successful collaboration between agents. By sharing information and working together, agents are able to provide better service to their clients and achieve greater success in their careers.""""</s>'

**Failure of HyDE Transform**


1.   HyDE may mislead when query can be mis-interpreted without context.
2.   HyDE may bias open-ended queries

